In [1]:
!pip install scikit-learn

  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


You should consider upgrading via the 'C:\Users\0\Documents\GitHub\Secuencias\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, MeanShift, MiniBatchKMeans, Birch, AffinityPropagation, AgglomerativeClustering
from sklearn import metrics

In [30]:
df_data = pd.read_csv("../../results_demo/absorption/NLP/bepler/bepler-absortion.csv")
df_data

,id,target,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,...,p_111,p_112,p_113,p_114,p_115,p_116,p_117,p_118,p_119,p_120
0,sequence_40826,539,0.1275,0.0336,0.0268,0.0302,0.0000,0.0268,0.0302,0.0570,...,-2.8427,-1.2708,-0.3906,-0.0434,-1.7144,-0.1575,-0.0376,-0.3315,-0.2037,2.8835
1,sequence_40787,510,0.1309,0.0336,0.0268,0.0302,0.0000,0.0268,0.0302,0.0570,...,-2.8364,-1.2318,-0.3848,-0.0487,-1.7134,-0.1354,-0.0321,-0.3253,-0.2264,2.8325
2,sequence_40789,510,0.1275,0.0336,0.0268,0.0302,0.0000,0.0268,0.0302,0.0604,...,-2.8511,-1.2120,-0.4116,-0.0691,-1.7377,-0.0950,-0.0326,-0.3248,-0.2055,2.8375
3,sequence_40794,519,0.1275,0.0369,0.0268,0.0302,0.0000,0.0268,0.0302,0.0570,...,-2.8301,-1.2142,-0.4579,-0.0766,-1.7762,-0.0257,-0.0328,-0.3276,-0.2366,2.8518
4,sequence_40806,525,0.1309,0.0336,0.0268,0.0302,0.0000,0.0268,0.0302,0.0570,...,-2.8361,-1.2607,-0.4201,-0.0540,-1.7401,-0.1217,-0.0339,-0.3242,-0.2101,2.8449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,sequence_40795,562,0.1275,0.0336,0.0268,0.0302,0.0034,0.0268,0.0302,0.0570,...,-2.8248,-1.2742,-0.3833,-0.0325,-1.7116,-0.1763,-0.0210,-0.3231,-0.2080,2.8842
77,sequence_40866,620,0.1242,0.0336,0.0268,0.0268,0.0034,0.0268,0.0336,0.0570,...,-2.8455,-1.2491,-0.3865,-0.0424,-1.6910,-0.1050,-0.0241,-0.2965,-0.1698,2.8735
78,sequence_40799,597,0.1242,0.0336,0.0268,0.0302,0.0034,0.0268,0.0302,0.0570,...,-2.8890,-1.2626,-0.3630,-0.0460,-1.6751,-0.1030,-0.0269,-0.2975,-0.1892,2.8983
79,sequence_40788,510,0.1242,0.0336,0.0268,0.0302,0.0034,0.0268,0.0302,0.0604,...,-2.8783,-1.1861,-0.3999,-0.0667,-1.7065,-0.0575,-0.0251,-0.3028,-0.2008,2.8420


In [31]:
def get_performance_clustering(data, labels):
    siluetas = metrics.silhouette_score(data, labels, metric='euclidean')
    calinski = metrics.calinski_harabasz_score(data, labels)
    davies = metrics.davies_bouldin_score(data, labels)

    return siluetas, calinski, davies

In [32]:
#ignoramos y removemos las columnas que no ocuparemos para el clustering
ignore_columns = pd.DataFrame()
ignore_columns['id'] = df_data['id']
ignore_columns['target'] = df_data['target']

df_data = df_data.drop(columns=['id', 'target'])

In [33]:
#exploramos KNN
matrix_result = []
for k in range(2, 30):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_data)
    siluetas, calinski, davies = get_performance_clustering(df_data, kmeans.labels_)
    row = ["k-means-{}".format(k), siluetas, calinski, davies]
    matrix_result.append(row)

df_explore = pd.DataFrame(matrix_result, columns=['strategy', 'siluetas', 'calinski', 'davies'])
df_explore.to_csv("../../results_demo/absorption/NLP/bepler/results_kmeans.csv")

In [34]:
#seleccionamos el mejor
import numpy as np

highest_siluetas = np.max(df_explore['siluetas'])
highest_calinski = np.max(df_explore['calinski'])

# filtramos
df_filter_by_siluetas = df_explore.loc[df_explore['siluetas'] >= highest_siluetas]
df_filter_by_calinski = df_explore.loc[df_explore['calinski'] >= highest_calinski]

df_concat = pd.concat([df_filter_by_siluetas, df_filter_by_calinski])
strategies = df_concat['strategy'].unique()
strategies


array(['k-means-3'], dtype=object)

In [35]:
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(df_data)
ignore_columns['labels'] = kmeans.labels_

ignore_columns.to_csv("../../results_demo/absorption/NLP/bepler/unsupervised_clustering_sequences_kmeans.csv")

In [48]:
#exploramos BIRCH
matrix_result = []
for k in range(2, 30):
    birch = Birch(n_clusters=k,threshold=0.1)
    birch.fit(df_data)
    siluetas, calinski, davies = get_performance_clustering(df_data, birch.labels_)
    row = ["birch-{}".format(k), siluetas, calinski, davies]
    matrix_result.append(row)

df_explore = pd.DataFrame(matrix_result, columns=['strategy', 'siluetas', 'calinski', 'davies'])
df_explore.to_csv("../../results_demo/absorption/NLP/bepler/results_birch.csv")

In [49]:
#seleccionamos el mejor
import numpy as np

highest_siluetas = np.max(df_explore['siluetas'])
highest_calinski = np.max(df_explore['calinski'])

# filtramos
df_filter_by_siluetas = df_explore.loc[df_explore['siluetas'] >= highest_siluetas]
df_filter_by_calinski = df_explore.loc[df_explore['calinski'] >= highest_calinski]

df_concat = pd.concat([df_filter_by_siluetas, df_filter_by_calinski])
strategies = df_concat['strategy'].unique()
strategies

array(['birch-3'], dtype=object)

In [51]:
birch = Birch(n_clusters=3,threshold=0.1)
birch.fit(df_data)
ignore_columns['labels'] = birch.labels_

ignore_columns.to_csv("../../results_demo/absorption/NLP/bepler/unsupervised_clustering_sequences_birch.csv")

In [52]:
#exploramos Aglomerative
matrix_result = []
for k in range(2, 30):
    aglomerative = AgglomerativeClustering(n_clusters=k)
    aglomerative.fit(df_data)
    siluetas, calinski, davies = get_performance_clustering(df_data, aglomerative.labels_)
    row = ["aglomerative-{}".format(k), siluetas, calinski, davies]
    matrix_result.append(row)

df_explore = pd.DataFrame(matrix_result, columns=['strategy', 'siluetas', 'calinski', 'davies'])
df_explore.to_csv("../../results_demo/absorption/NLP/bepler/results_aglomerative.csv")

In [53]:
#seleccionamos el mejor
import numpy as np

highest_siluetas = np.max(df_explore['siluetas'])
highest_calinski = np.max(df_explore['calinski'])

# filtramos
df_filter_by_siluetas = df_explore.loc[df_explore['siluetas'] >= highest_siluetas]
df_filter_by_calinski = df_explore.loc[df_explore['calinski'] >= highest_calinski]

df_concat = pd.concat([df_filter_by_siluetas, df_filter_by_calinski])
strategies = df_concat['strategy'].unique()
strategies


array(['aglomerative-3'], dtype=object)

In [54]:
# en este caso salio el mejor k-means=2
aglomerative = AgglomerativeClustering(n_clusters=3)
aglomerative.fit(df_data)
ignore_columns['labels'] = aglomerative.labels_

ignore_columns.to_csv("../../results_demo/absorption/NLP/bepler/unsupervised_clustering_sequences_aglomerative.csv")